# Imports

In [1]:
import json
import os
import bz2
import io
from bz2 import BZ2File
import pandas as pd
import numpy as np

# Read dataset

In [2]:
input_dir = "data" # create folder 'data' containing .jsonl.bz2 file and update with your path 

In [3]:
# a helper function to get the lines from am archive
def read_jsonlines(bz2_file):
    text = bz2_file.read().decode('utf-8')
    for line in text.split('\n'):
        if line != '':
            yield line

In [4]:
archives = os.listdir(input_dir)

# Extract articles

In [5]:
journal_ = []
date_ = []
page_ = []
totalErrors = 0

In [6]:
%%time
for archive in archives:
    
    # take only the transformed archives
    # open the archive
    f = BZ2File(os.path.join(input_dir, archive), 'r')
    articles = list(read_jsonlines(f))
    # get the list of articles it contains (= a json object on each line)
    print(archive, ':', len(articles), 'articles à extraire')
        
    # load the first 100 articles as json and access their attributes
    totalErrors = 0
    for a in articles:
        try:
            json_article = json.loads(a)
            fulltext = str(json_article["ft"])
            if "nucleaire" in fulltext or "nucléaire" in fulltext:
                # decode the json string into an object (dict)
                journal_.append(str(json_article["id"])[:3])
                date_.append(str(json_article["id"])[4:14])
                page_.append(str(json_article["pp"])[1:-1])
        except:
            totalErrors += 1
    print("total errors", totalErrors)

GDL-1907.jsonl.bz2 : 28146 articles à extraire
total errors 41
GDL-1997.jsonl.bz2 : 74031 articles à extraire
total errors 8337
JDG-1927.jsonl.bz2 : 35000 articles à extraire
total errors 260
JDG-1906.jsonl.bz2 : 18936 articles à extraire
total errors 147
JDG-1952.jsonl.bz2 : 37481 articles à extraire
total errors 1107
GDL-1924.jsonl.bz2 : 33627 articles à extraire
total errors 13
GDL-1902.jsonl.bz2 : 27702 articles à extraire
total errors 25
JDG-1933.jsonl.bz2 : 32383 articles à extraire
total errors 410
GDL-1949.jsonl.bz2 : 38638 articles à extraire
total errors 479
GDL-1962.jsonl.bz2 : 51816 articles à extraire
total errors 3272
JDG-1972.jsonl.bz2 : 45846 articles à extraire
total errors 1353
GDL-1958.jsonl.bz2 : 45144 articles à extraire
total errors 1328
JDG-1987.jsonl.bz2 : 68753 articles à extraire
total errors 9907
GDL-1933.jsonl.bz2 : 31841 articles à extraire
total errors 490
GDL-1954.jsonl.bz2 : 40889 articles à extraire
total errors 773
JDG-1966.jsonl.bz2 : 46881 articles à

total errors 1299
JDG-1982.jsonl.bz2 : 55201 articles à extraire
total errors 5048
JDG-1980.jsonl.bz2 : 49477 articles à extraire
total errors 4518
GDL-1981.jsonl.bz2 : 55490 articles à extraire
total errors 4979
GDL-1925.jsonl.bz2 : 33319 articles à extraire
total errors 175
GDL-1986.jsonl.bz2 : 65316 articles à extraire
total errors 8626
JDG-1921.jsonl.bz2 : 27071 articles à extraire
total errors 186
GDL-1919.jsonl.bz2 : 42037 articles à extraire
total errors 25
JDG-1912.jsonl.bz2 : 24460 articles à extraire
total errors 110
JDG-1992.jsonl.bz2 : 71017 articles à extraire
total errors 8982
JDG-1907.jsonl.bz2 : 19821 articles à extraire
total errors 124
JDG-1977.jsonl.bz2 : 46117 articles à extraire
total errors 1957
GDL-1928.jsonl.bz2 : 33665 articles à extraire
total errors 196
GDL-1941.jsonl.bz2 : 26820 articles à extraire
total errors 436
GDL-1993.jsonl.bz2 : 65764 articles à extraire
total errors 8507
JDG-1941.jsonl.bz2 : 25230 articles à extraire
total errors 718
JDG-1964.jsonl.b

# Create a pandas DataFrame

In [ ]:
dec = json.loads(articles[0])
dec["ft"]

In [7]:
df = pd.DataFrame.from_dict(
    {
        'journal': journal_,
        'date': date_,
        'page': page_,
    })

In [8]:
df.head()

,journal,date,page
0,GDL,1997-10-28,4
1,GDL,1997-10-10,12
2,GDL,1997-10-10,32
3,GDL,1997-10-15,5
4,GDL,1997-10-11,14


In [9]:
len(df)

50737

# Clean and enrich the DataFrame 

In [11]:
def measure_articles(df):
    
    lengths = []
    
    for ind, row in df.iterrows():
        lengths.append(len(row['text']))
        
    return lengths

df['length'] = measure_articles(df)

KeyError: 'text'

In [11]:
import re
def handle_multiple_pages(df):
    
    page, ppage = [], []
    
    for ind, row in df.iterrows():
        
        found = re.findall('([0-9]+)', row['page'])
        if len(found) > 1:
            page.append(found[0])
            ppage.append(found[1])
        else:
            page.append(row['page'])
            ppage.append(np.nan)
        
    return page, ppage
    
page, ppage = handle_multiple_pages(df)
df['page'] = page
df['ppage'] = ppage

In [12]:
df.head()

,journal,date,page,text,length,ppage
0,GDL,1969-07-07,1,Privée de gouvernement l'Italie est inquiète A...,3667,NaN
1,GDL,1969-07-07,2,*• : & m wim W ^ i''***» A Cham on s'attaque a...,875,NaN
2,GDL,1969-07-07,3,Cent évêques européens aujourd'hui à Coire Com...,1063,NaN
3,GDL,1969-07-07,4,Meta Antenen : record du monde AVEC 5046 POINT...,3470,NaN
4,GDL,1969-07-07,5,GAÉTAN GANY A ESSAYÉ POUR VOUS Une Opel dans l...,9429,NaN


In [10]:
# Jeter les articles vides ou ne contenant que quelques caractères (p.ex titre des rubriques)
df = df[df['length'] > 50]

KeyError: 'length'

In [12]:
# Sauvegarder l'index
df['id'] = df.index

In [13]:
# Formater les types
df['date'] = pd.to_datetime(df['date'])
df['journal'] = df['journal'].astype('category')
df['page'] = df['page'].astype('int')
# df['ppage'] = df['ppage'].astype('float')
# df['text'] = df['text'].astype('str')

ValueError: invalid literal for int() with base 10: '3, 4'

In [14]:
df = df[['id', 'journal', 'date', 'page']]

In [15]:
df.to_json('cleaned_pages.json.bz2', compression = 'bz2')

In [18]:
df_lengths = df['length'].value_counts()

In [19]:
# Creating txt file
txt = open('data.txt', 'w+')
for ind, row in df.iterrows():
    if "nucleaire" in row['text'] or "nucléaire" in row['text']:
        date = str(row['date'])[0:10]
        txt.write('**** *'), txt.write(str(ind))
        txt.write(' *'), txt.write(str(row['page']))
        txt.write(' *'), txt.write(row['journal'])
        txt.write(' *'), txt.write(date), txt.write('\n')
        txt.write(row['text']), txt.write('\n')

txt.close()